In [1]:
import pandas as pd

import torch
import numpy as np

In [2]:
def z_norm(data):
    std = data.std(0).unsqueeze(0)
    std = torch.where(std == 0, torch.tensor(1, dtype=torch.float32).cpu(), std)
    return (data - data.mean(0).unsqueeze(0)) / std


def inverse_z_norm(normalized_data, original_data):
    std = original_data.std(0).unsqueeze(0)
    std = torch.where(std == 0, torch.tensor(1, dtype=torch.float32).cpu(), std)
    mean = original_data.mean(0).unsqueeze(0)
    
    return (normalized_data * std) + mean

In [3]:
formatted = pd.read_csv('formatted_transactions.csv')

In [8]:
dataset = torch.load('processed/data.pt')

In [5]:
new_data = torch.load('newest/data.pt')

In [12]:
dataset['train']#[('node', 'to', 'node')].edge_id

HeteroData(
  node={
    x=[119393, 1],
    num_nodes=119393,
  },
  (node, to, node)={
    edge_index=[2, 64153],
    edge_attr=[64153, 4],
    y=[64153],
    timestamps=[64153],
    split_mask=[64153],
  },
  (node, rev_to, node)={
    edge_index=[2, 64153],
    edge_attr=[64153, 4],
  }
)

In [14]:
# Extract the full edge index for a specific edge type, for example, 'transaction'
edge_index = dataset['train'][('node', 'to', 'node')].edge_index
edge_labels = dataset['train'][('node', 'to', 'node')].y

# Define your own batch size for evaluation:
eval_batch_size = 2048  # Set this as suits memory constraints

num_edges = edge_index.size(1)
num_batches = (num_edges + eval_batch_size - 1) // eval_batch_size

all_predictions = []


with torch.no_grad():
    for batch in range(num_batches):
        start = batch * eval_batch_size
        end = min((batch + 1) * eval_batch_size, num_edges)

        # Slice the full edge index for your batch
        batch_edge_index = edge_index[:, start:end]
        print(batch_edge_index)

        # Construct a subgraph for the current edge batch.
        # This may involve extracting associated node features and other attributes.
        # You might need to write a helper function for this extraction.
#         batch.split = split
#         batch.to(torch.device(cfg.device))

#         subgraph = extract_subgraph(data, batch_edge_index)
#         subgraph = subgraph.to(cfg.device)

#         # Forward pass the subgraph
#         pred, true, edge_id = model(batch, return_edges=True)
#         batch_preds = out['transaction']  # Adjust as necessary
#         all_predictions.append(batch_preds.cpu())
        
# # Concatenate all predictions and compute metrics over the entire set.
# all_predictions = torch.cat(all_predictions, dim=0)

tensor([[ 3225,  4321,  4744,  ...,  2071, 94494, 94816],
        [ 3225,  4321,  4745,  ...,  2071, 94494, 94817]])
tensor([[94874, 95834, 95859,  ..., 52022, 52328, 52708],
        [94874, 95834, 95859,  ..., 52022, 52328, 52708]])
tensor([[32226, 47152, 53008,  ...,  1967,  2697,  4899],
        [32226, 47152, 53008,  ...,  1968,  2697,  4899]])
tensor([[ 5214,  8465,  8975,  ..., 38432, 20184, 63458],
        [ 5215,  8465,  8976,  ..., 38433, 20184, 63459]])
tensor([[ 65321,  78299,  84740,  ...,  59737, 114361, 114474],
        [ 65321,  78300,  84740,  ...,  87078, 114361, 114475]])
tensor([[29232, 43022, 48617,  ...,   173, 15554, 27821],
        [29233, 43022, 48618,  ...,   173, 15555, 27822]])
tensor([[49119, 58181, 68522,  ..., 51722, 26275, 90497],
        [49120,   922, 68522,  ..., 58618,  7270, 90497]])
tensor([[ 99490,  40971, 106635,  ...,  98649, 101168, 101173],
        [ 99491, 103586, 106635,  ...,  91191,  52756, 101173]])
tensor([[105762, 116569, 116939,  ..., 1

In [47]:
formatted['Timestamp']

0       -345590
1       -345590
2       -345590
3       -345590
4       -345590
          ...  
99995    818170
99996    822490
99997    828610
99998    832450
99999    972010
Name: Timestamp, Length: 100000, dtype: int64

In [7]:
len(data['train'][('node', 'to', 'node')].edge_index[0])

64153

In [20]:
data['train']#[('node', 'rev_to', 'node')]

HeteroData(
  node={
    x=[119393, 1],
    num_nodes=119393,
  },
  (node, to, node)={
    edge_index=[2, 64153],
    edge_attr=[64153, 4],
    y=[64153],
    timestamps=[64153],
    split_mask=[64153],
  },
  (node, rev_to, node)={
    edge_index=[2, 64153],
    edge_attr=[64153, 4],
  }
)

In [39]:
data['train'][('node', 'to', 'node')].edge_index.unique()

tensor([     0,      1,      3,  ..., 119390, 119391, 119392])

In [40]:
data['val'][('node', 'to', 'node')].edge_index.unique()

tensor([     0,      1,      2,  ..., 119390, 119391, 119392])

In [43]:
data['train'][('node', 'to', 'node')].timestamps

tensor([     0.,      0.,      0.,  ..., 518340., 518340., 518340.])

In [6]:
data= pd.read_csv('bank_0/dataset0.csv')
predictions = pd.read_csv('bank_0/predictions.csv')
formatted = pd.read_csv('bank_0/formatted_transactions_0.csv')

In [7]:
formatted

,EdgeID,from_id,to_id,Timestamp,Amount Sent,Sent Currency,Amount Received,Received Currency,Payment Format,Is Laundering
0,24,32,32,10,19.18,0,19.18,0,0,0
1,58,58,59,10,1199.83,0,1199.83,0,4,0
2,110,104,104,10,395667.01,0,395667.01,0,0,0
3,117,109,110,10,227247.42,0,227247.42,0,1,0
4,130,118,118,10,215054.84,0,215054.84,0,0,0
...,...,...,...,...,...,...,...,...,...,...
1207493,1163065,50,29415,13102450,635.62,1,635.62,1,3,1
1207494,1165980,29489,15187,13275250,11188.76,0,11188.76,0,3,1
1207495,1163066,50,50,13375930,19191.22,0,16377.79,1,3,0
1207496,1163067,50,29416,13375930,16377.79,1,16377.79,1,3,1


In [8]:
data.index = data.index.astype(int)
data_reset = data.reset_index()
data_reset

,index,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
0,0,2022/08/01 00:07,0,8009B22F0,0,8009B22F0,121.98,US Dollar,121.98,US Dollar,Reinvestment,0
1,1,2022/08/01 00:22,0,800815DE0,0,800815DE0,23.40,US Dollar,23.40,US Dollar,Reinvestment,0
2,2,2022/08/01 00:17,0,800A39060,0,800A39060,34776847.10,US Dollar,34776847.10,US Dollar,Reinvestment,0
3,3,2022/08/01 00:14,0,800A39060,2310,800A39B80,4332.00,US Dollar,4332.00,US Dollar,Cheque,0
4,4,2022/08/01 00:04,31620,800BA9350,0,800ABF260,0.01,US Dollar,0.01,US Dollar,Cheque,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1207493,1207493,2022/11/05 23:36,0,80053E520,10057,83F3F2850,3014.21,US Dollar,3014.21,US Dollar,ACH,0
1207494,1207494,2022/11/05 23:48,0,804228F70,0,804228F70,14465.55,Yuan,2159.81,US Dollar,ACH,0
1207495,1207495,2022/11/05 23:48,0,804228F70,233506,819288650,14465.55,Yuan,14465.55,Yuan,Cheque,0
1207496,1207496,2022/11/05 23:31,0,804228F70,0,804228F70,9281.54,Yuan,1385.80,US Dollar,ACH,0


In [9]:
predictions['Edge_ID'] = predictions['Edge_ID'].astype(int)
predictions['Time_stamp'] = predictions['Time_stamp'].astype(int)

KeyError: 'Time_stamp'

In [6]:
df_merged =  data.merge(formatted, left_index=True, right_on='EdgeID')

df_merged = df_merged.merge(predictions, left_index=True, right_on='Edge_ID')

In [14]:
df_merged['predicted_label'] = 1 / (1 + np.exp(-df_merged['Prediction'])) #torch.sigmoid(df_merged['Prediction'])

df_merged.columns

Index(['Timestamp_x', 'From Bank', 'Account', 'To Bank', 'Account.1',
       'Amount Received_x', 'Receiving Currency', 'Amount Paid',
       'Payment Currency', 'Payment Format_x', 'Is Laundering_x', 'EdgeID',
       'from_id', 'to_id', 'Timestamp_y', 'Amount Sent', 'Sent Currency',
       'Amount Received_y', 'Received Currency', 'Payment Format_y',
       'Is Laundering_y', 'Edge_ID', 'Prediction', 'True_Label', 'Time_stamp',
       'Timestamp_new', 'predicted_label'],
      dtype='object')

In [8]:
df_merged[['Time_stamp','Timestamp_y']] #= #[df_merged['Is Laundering_x']!=df_merged['True_Label']]
df_merged['Timestamp_new'] = df_merged['Timestamp_y']-10

df_merged[['Time_stamp','Timestamp_y','Timestamp_new']]

,Time_stamp,Timestamp_y,Timestamp_new
61008,7547940.0,2340550,2340540
26324,7485780.0,2429050,2429040
11817,7146660.0,2500810,2500800
6328,7933320.0,2500810,2500800
10531,6813300.0,2566330,2566320
...,...,...,...
19741,8382900.0,11377690,11377680
10835,8368440.0,8382610,8382600
72725,8371200.0,8382850,8382840
34648,8371860.0,8383810,8383800


In [60]:
df_merged[df_merged['Timestamp_new']!=df_merged['Time_stamp']]

,index,Timestamp_x,From Bank,Account,To Bank,Account.1,Amount Received_x,Receiving Currency,Amount Paid,Payment Currency,...,Sent Currency,Amount Received_y,Received Currency,Payment Format_y,Is Laundering_y,Edge_ID,Prediction,True_Label,Time_stamp,Timestamp_new


In [62]:
df_merged[df_merged['Is Laundering_y']==df_merged['True_Label']]

,index,Timestamp_x,From Bank,Account,To Bank,Account.1,Amount Received_x,Receiving Currency,Amount Paid,Payment Currency,...,Sent Currency,Amount Received_y,Received Currency,Payment Format_y,Is Laundering_y,Edge_ID,Prediction,True_Label,Time_stamp,Timestamp_new
0,340148,2022/10/27 08:39,14,8001A7E40,0,80013C630,13379.68,US Dollar,13379.68,US Dollar,...,0,13379.68,0,3,1,340148,0.219746,1,7547940,7547940
1,346963,2022/10/26 15:23,11,800074100,0,8001D5A80,42344223.96,US Dollar,42344223.96,US Dollar,...,0,42344223.96,0,3,1,346963,0.092474,1,7485780,7485780
2,358077,2022/10/22 17:11,9340,8043F93F0,0,800E2E930,18964.75,US Dollar,18964.75,US Dollar,...,0,18964.75,0,3,1,358077,0.153107,1,7146660,7146660
3,358078,2022/10/31 18:42,9340,8043F93F0,0,8005BAD80,10046.37,US Dollar,10046.37,US Dollar,...,0,10046.37,0,3,1,358078,0.030533,1,7933320,7933320
4,368484,2022/10/18 20:35,1827,800FED3F0,0,800E6EEF0,15631.26,US Dollar,15631.26,US Dollar,...,0,15631.26,0,3,1,368484,0.102941,1,6813300,6813300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76974,1207465,2022/11/05 23:38,0,81C1CD160,18170,81C1CD1B0,15497.98,US Dollar,15497.98,US Dollar,...,0,15497.98,0,1,0,1207465,0.011737,0,8383080,8383080
76975,1207469,2022/11/05 23:55,118303,80C087090,0,81CA370D0,16.23,US Dollar,16.23,US Dollar,...,0,16.23,0,3,0,1207469,0.021208,0,8384100,8384100
76976,1207473,2022/11/05 23:48,0,800249E50,240503,820183C30,177.62,US Dollar,177.62,US Dollar,...,0,177.62,0,4,0,1207473,-0.072041,0,8383680,8383680
76977,1207478,2022/11/05 23:34,0,81A3F44F0,152270,828271A30,3405.44,US Dollar,3405.44,US Dollar,...,0,3405.44,0,2,0,1207478,0.034504,0,8382840,8382840


In [65]:
df_merged[df_merged['EdgeID']!=df_merged['Edge_ID']]

,index,Timestamp_x,From Bank,Account,To Bank,Account.1,Amount Received_x,Receiving Currency,Amount Paid,Payment Currency,...,Sent Currency,Amount Received_y,Received Currency,Payment Format_y,Is Laundering_y,Edge_ID,Prediction,True_Label,Time_stamp,Timestamp_new


In [66]:
df_merged['Edge_ID'].unique

<bound method Series.unique of 0         340148
1         346963
2         358077
3         358078
4         368484
          ...   
76974    1207465
76975    1207469
76976    1207473
76977    1207478
76978    1207484
Name: Edge_ID, Length: 76979, dtype: int32>

In [20]:
print(set(predictions['Edge_ID']) - set(data.index))

set()


In [21]:
print("Data index unique:", data.index.is_unique)
print("Predictions Edge_ID unique:", predictions['Edge_ID'].is_unique)

Data index unique: True
Predictions Edge_ID unique: True


In [2]:
auth = pd.read_csv('alldata_authorities.csv')

auth

,Timestamp_x,From Bank,Account,To Bank,Account.1,Amount Received_x,Receiving Currency,Amount Paid,Payment Currency,Payment Format_x,...,Amount Received_y,Received Currency,Payment Format_y,Is Laundering_y,Edge_ID,Prediction,True_Label,Time_stamp,Timestamp_new,predicted_label
0,2022/08/20 08:00,110749,806C07F10,11,80421DC90,190.51,Euro,190.51,Euro,ACH,...,190.51,1,3,0,271246.0,-0.516715,1,NaN,NaN,0.373621
1,2022/08/20 08:00,110749,806C07F10,11,80421DC90,10783.56,Euro,10783.56,Euro,ACH,...,10783.56,1,3,1,271247.0,-0.114300,1,NaN,NaN,0.471456
2,2022/08/18 22:26,11,8060C8630,1512,80B879E80,3008.99,Euro,3008.99,Euro,Cheque,...,3008.99,1,1,0,246606.0,0.460552,0,NaN,NaN,0.613145
3,2022/08/18 22:26,11,8001BA510,18607,81C832B70,1944.05,Euro,1944.05,Euro,Cash,...,1944.05,1,4,0,246607.0,0.460234,1,NaN,NaN,0.613070
4,2022/10/19 08:39,11,800074100,11,800074100,478.63,Euro,560.86,US Dollar,ACH,...,478.63,1,3,0,1057760.0,-0.023668,0,NaN,NaN,0.494083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166691,2022/11/05 23:52,12,80F1ECCF0,37220,80FA218D0,10341.00,Yuan,10341.00,Yuan,Credit Card,...,10341.00,2,2,0,1299320.0,-2.496451,0,NaN,NaN,0.076107
166692,2022/11/05 23:44,12,80CE69ED0,36395,80FAE4680,8905.75,Yuan,8905.75,Yuan,Cheque,...,8905.75,2,1,0,1299290.0,0.039634,0,NaN,NaN,0.509907
166693,2022/11/05 23:53,236256,830AF92B0,12,832476EF0,17773.95,Yuan,17773.95,Yuan,Cheque,...,17773.95,2,1,0,1299325.0,0.084643,0,NaN,NaN,0.521148
166694,2022/11/05 23:31,236256,830AF92B0,12,832476EF0,1428.28,Yuan,1428.28,Yuan,Wire,...,1428.28,2,4,0,1299241.0,0.025040,0,NaN,NaN,0.506260


In [7]:
threshold = 0.5

auth[auth['predicted_label']>threshold]

,Timestamp_x,From Bank,Account,To Bank,Account.1,Amount Received_x,Receiving Currency,Amount Paid,Payment Currency,Payment Format_x,...,Amount Received_y,Received Currency,Payment Format_y,Is Laundering_y,Edge_ID,Prediction,True_Label,Time_stamp,Timestamp_new,predicted_label
2,2022/08/18 22:26,11,8060C8630,1512,80B879E80,3008.99,Euro,3008.99,Euro,Cheque,...,3008.99,1,1,0,246606.0,0.460552,0,NaN,NaN,0.613145
3,2022/08/18 22:26,11,8001BA510,18607,81C832B70,1944.05,Euro,1944.05,Euro,Cash,...,1944.05,1,4,0,246607.0,0.460234,1,NaN,NaN,0.613070
5,2022/10/19 08:39,11,800074100,11,800138210,478.63,Euro,478.63,Euro,ACH,...,478.63,1,3,1,1057761.0,0.007722,0,NaN,NaN,0.501931
9,2022/08/20 07:58,11,80040EE80,12,8026E8B30,299968.01,Euro,299968.01,Euro,ACH,...,299968.01,1,3,0,271242.0,0.003566,0,NaN,NaN,0.500891
10,2022/08/20 07:59,11,801ECC980,126145,80BC4DA80,23968.81,Euro,23968.81,Euro,Credit Card,...,23968.81,1,2,0,271245.0,0.003423,0,NaN,NaN,0.500856
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166688,2022/11/05 23:46,12,800A808D0,135161,80D5256C0,35879.41,Yuan,35879.41,Yuan,Cash,...,35879.41,2,3,0,1299298.0,0.001024,0,NaN,NaN,0.500256
166689,2022/11/05 23:32,1046,8020B4DC0,12,80F42AFC0,87519.37,Yuan,87519.37,Yuan,Credit Card,...,87519.37,2,2,0,1299246.0,0.000272,0,NaN,NaN,0.500068
166692,2022/11/05 23:44,12,80CE69ED0,36395,80FAE4680,8905.75,Yuan,8905.75,Yuan,Cheque,...,8905.75,2,1,0,1299290.0,0.039634,0,NaN,NaN,0.509907
166693,2022/11/05 23:53,236256,830AF92B0,12,832476EF0,17773.95,Yuan,17773.95,Yuan,Cheque,...,17773.95,2,1,0,1299325.0,0.084643,0,NaN,NaN,0.521148


In [9]:
auth.shape()

TypeError: 'tuple' object is not callable

In [35]:
data= pd.read_csv(f'bank_0/dataset0.csv')
predictions = pd.read_csv(f'bank_0/predictions.csv')
formatted = pd.read_csv(f'bank_0/formatted_transactions.csv')
df_merged =  data.merge(formatted, left_index=True, right_on='EdgeID')
df_merged = df_merged.merge(predictions, left_on='EdgeID', right_on='Edge_ID')

#df_merged['predicted_label'] = 1 / (1 + np.exp(-df_merged['Prediction']))

#df = pd.concat([df, df_merged], ignore_index=True)

In [39]:
df_merged[df_merged['Is Laundering_y']!=df_merged['True_Label']]

,Timestamp_x,From Bank,Account,To Bank,Account.1,Amount Received_x,Receiving Currency,Amount Paid,Payment Currency,Payment Format_x,...,Timestamp_y,Amount Sent,Sent Currency,Amount Received_y,Received Currency,Payment Format_y,Is Laundering_y,Edge_ID,Prediction,True_Label


In [20]:
formatted

,EdgeID,from_id,to_id,Timestamp,Amount Sent,Sent Currency,Amount Received,Received Currency,Payment Format,Is Laundering
0,24,32,32,10,19.18,0,19.18,0,0,0
1,58,58,59,10,1199.83,0,1199.83,0,4,0
2,110,104,104,10,395667.01,0,395667.01,0,0,0
3,117,109,110,10,227247.42,0,227247.42,0,1,0
4,130,118,118,10,215054.84,0,215054.84,0,0,0
...,...,...,...,...,...,...,...,...,...,...
1207493,1163065,50,29415,13102450,635.62,1,635.62,1,3,1
1207494,1165980,29489,15187,13275250,11188.76,0,11188.76,0,3,1
1207495,1163066,50,50,13375930,19191.22,0,16377.79,1,3,0
1207496,1163067,50,29416,13375930,16377.79,1,16377.79,1,3,1


In [32]:
#df_merged[df_merged['Is Laundering_x']!=df_merged['True_Label']]
df_merged[df_merged['EdgeID']!=df_merged['Edge_ID']][['Edge_ID','EdgeID']]

KeyError: 'Edge_ID'